In [ ]:
# Install dependencies
!pip install opencv-python-headless moviepy

import cv2
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

# Download YOLO weights and config files
!wget https://pjreddie.com/media/files/yolov3.weights -O yolov3.weights
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg -O yolov3.cfg
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names -O coco.names

--2024-11-16 08:53:31--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  21.2MB/s    in 11s     

2024-11-16 08:53:43 (21.6 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]

--2024-11-16 08:53:43--  https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8342 (8.1K) [text/plain]
Saving to: ‘yolov3.cfg’

yolov3.cfg          100%[===================>]   8.15K  --.-KB/s    in 0.001s  

2

In [80]:
def detect_yolo(input_image_path):
    # Load YOLO model
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    # Set up detection layers
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

    # Load your image
    image = cv2.imread(input_image_path)
    height, width, channels = image.shape
    # Prepare image for YOLO
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    # Extract bounding boxes and class IDs
    class_ids = []
    confidences = []
    boxes = []
    results = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Confidence threshold
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
                results.append({
                    "class": classes[class_id],
                    "confidence": float(confidence),
                    "x": x,
                    "y": y,
                    "width": w,
                    "height": h
                })
    return results

In [81]:
# Step 2: Define Non-Maximum Suppression (NMS) Function
def non_max_suppression(boxes, confidences, iou_threshold=0.4):
    # Convert boxes to x1, y1, x2, y2 format
    converted_boxes = np.array([[x, y, x + w, y + h] for x, y, w, h in boxes])
    confidences = np.array(confidences)

    # Perform NMS using OpenCV
    indices = cv2.dnn.NMSBoxes(
        bboxes=converted_boxes.tolist(),
        scores=confidences.tolist(),
        score_threshold=0.5,
        nms_threshold=iou_threshold
    )
    return indices.flatten() if len(indices) > 0 else []

In [82]:
# Step 3: Apply NMS to Filter Boxes
def filter_boxes(results, iou_threshold=0.4):
    # Extract bounding box coordinates and confidence scores
    boxes = [[item["x"], item["y"], item["width"], item["height"]] for item in results]
    confidences = [item["confidence"] for item in results]

    # Perform NMS
    nms_indices = non_max_suppression(boxes, confidences, iou_threshold)

    # Filter the results
    return [results[i] for i in nms_indices]

In [83]:
# Step 4: Draw Borders from Filtered JSON
def draw_boxes_from_json(input_image_path, json_data):
    # Load the image
    image = cv2.imread(input_image_path)
    base_name = input_image_path.split("/")[3][0:-4]

    # Draw each box
    for box in json_data:
        x, y, w, h = box["x"], box["y"], box["width"], box["height"]
        label = box["class"]
        confidence = box["confidence"]
        color = (0, 255, 0)  # Green for vehicles
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
        text = f"{label} {confidence:.2f}"
        cv2.putText(image, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    # Save the filtered JSON data to a file
    output_json_path = f"/content/borders/{base_name}.json"
    with open(output_json_path, "w") as json_file:
        json.dump(json_data, json_file, indent=4)
    print(f"Filtered JSON saved to {output_json_path}")

    # Save the output image
    cv2.imwrite(f"/content/outputs/{base_name}_bordered.jpg", image)
    print(f"Result saved to /content/outputs/{base_name}_bordered.jpg")

In [86]:
folder_path = "/content/originals/"
images = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]
for input_image_path in images:
    results = detect_yolo(input_image_path)
    # Filter bounding boxes using NMS
    filtered_data = filter_boxes(results, iou_threshold=0.4)

    # Draw and save the results on the image
    draw_boxes_from_json(input_image_path, filtered_data)

Filtered JSON saved to /content/borders/152.json
Result saved to /content/outputs/152_bordered.jpg
Filtered JSON saved to /content/borders/223.json
Result saved to /content/outputs/223_bordered.jpg
Filtered JSON saved to /content/borders/164.json
Result saved to /content/outputs/164_bordered.jpg
Filtered JSON saved to /content/borders/150.json
Result saved to /content/outputs/150_bordered.jpg
